In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# tale.txt 파일 읽기
txt_file_path = "/content/drive/MyDrive/Tale/tale.txt"
with open(txt_file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 일부분 출력
print(text.split('\n')[:5])

print(len(text.split('\n\n\n')))

['늑대가 양떼무리에서 떨어진 어린양을 만났어요.', '그래서 잡아먹기로 작정했지요.', '그래서 그렇듯 한 이야기를 하며 잡아먹을 생각을 했어요.', '늑대가 어린양에게 말했어요. "어이, 네가 작년에 나 욕했지."', '"진짜 아니에요..." 라며 어린양이 애처로운 목소리로 말했어요. "전 작년에 태어나지도 않은걸요…"']
202


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, PreTrainedTokenizerFast, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np

txt_file_path = "/content/drive/MyDrive/Tale/tale.txt"
with open(txt_file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 동화들로 분리 (\n\n\n 사용)
fairy_tales = text.split('\n\n\n')
print(f"총 {len(fairy_tales)}개의 동화를 읽었습니다.")

# 토크나이저 및 모델 초기화
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
model.config.pad_token_id = model.config.eos_token_id

# 데이터셋 분할 (학습 80%, 검증 20%)
np.random.seed(42)
indices = np.random.permutation(len(fairy_tales))
train_indices = indices[:int(0.8*len(indices))]  # 변경: train_idx -> train_indices
val_indices = indices[int(0.8*len(indices)):]  # 변경: val_idx -> val_indices
train_tales = [fairy_tales[i] for i in train_indices]
val_tales = [fairy_tales[i] for i in val_indices]

# 동화 데이터셋 클래스
class FairyTaleDataset(Dataset):
    def __init__(self, fairy_tales, tokenizer, max_length=1024):
        self.fairy_tales = fairy_tales
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.fairy_tales)

    def __getitem__(self, idx):
        tale = self.fairy_tales[idx]
        encoding = self.tokenizer(tale, truncation=True, max_length=self.max_length)
        return encoding

# 동적 배치 크기를 위한 데이터 콜레이터
class DynamicDataCollatorForLanguageModeling(DataCollatorForLanguageModeling):
    def __call__(self, examples):
        # 길이에 따라 정렬
        examples = sorted(examples, key=lambda x: len(x["input_ids"]), reverse=True)
        batch = super().__call__(examples)
        batch["labels"][batch["input_ids"] == self.tokenizer.pad_token_id] = -100
        return batch

# 데이터셋 및 데이터 콜레이터 설정
train_dataset = FairyTaleDataset(train_tales, tokenizer)
val_dataset = FairyTaleDataset(val_tales, tokenizer)
data_collator = DynamicDataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Tale/kogpt2_fairy_tales",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)
# Perplexity 계산 함수
def compute_perplexity(model, dataset, data_collator, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=data_collator, shuffle=False)
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            non_padded_tokens = torch.sum(attention_mask)
            total_loss += loss.item() * non_padded_tokens.item()
            total_tokens += non_padded_tokens.item()

    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    return perplexity

# Trainer 설정 및 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tuning 전 Perplexity 계산
pre_train_perplexity = compute_perplexity(model, val_dataset, data_collator, device)
print(f"Fine-tuning 전 Perplexity: {pre_train_perplexity:.2f}")

# 모델 학습
trainer.train()

# Fine-tuning 후 Perplexity 계산
post_train_perplexity = compute_perplexity(model, val_dataset, data_collator, device)
print(f"Fine-tuning 후 Perplexity: {post_train_perplexity:.2f}")

# 최종 모델 저장
trainer.save_model("/content/drive/MyDrive/Tale/fine_tuned_kogpt2_fairy_tales")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
총 202개의 동화를 읽었습니다.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Fine-tuning 전 Perplexity: 26.95


Epoch,Training Loss,Validation Loss
1,No log,3.195273
2,No log,3.151115
3,No log,3.143563
4,No log,3.176833
5,No log,3.221823
6,No log,3.243867
7,No log,3.282624
8,No log,3.293279
9,No log,3.309012
10,No log,3.314138


Fine-tuning 후 Perplexity: 28.56


In [ ]:
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM

# 토크나이저 및 모델 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

model_path = '/content/drive/MyDrive/Tale/fine_tuned_kogpt2_fairy_tales'  # 미세조정된 모델 경로
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 텍스트 생성 함수
def generate_fairy_tale(model, tokenizer, prompts, max_length=120, temperature=0.8, top_p=0.5, num_return_sequences=1):
    tale = ""
    for i, prompt in enumerate(prompts):
        if i > 0:
            tale += ". "
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            do_sample=True
        )
        generated_chapter = tokenizer.decode(output[0], skip_special_tokens=True)
        tale += generated_chapter.capitalize() + " 어요."

    return tale

# 고래 백경이 등장하는 각 챕터의 개요를 상상하여 미리 입력하기
chapter_prompts = [
    "한 바다에 작은 어부가 살고 있었어요. 그의 이름은 영식이었어요. 어느 날, 영식이는 바다에 나가서 고래를 잡기로 결심했어요.",
    "영식이는 고래를 만나게 되었어요. 그러나 그 고래는 사실 고래 백경이었어요. 고래 백경은 영식이에게 부탁을 했어요.",
    "영식이는 고래 백경의 부탁을 들어주었어요. 그러면서 고래 백경은 영식이와 함께 모험을 떠났어요.",
    "마지막으로, 영식이와 고래 백경은 함께 어려움을 극복하고 친구가 되었어요. 그리고 그들은 항상 서로를 기억하고 지켜줄 것을 약속했어요."
]

# 동화 생성 및 출력
generated_tale = generate_fairy_tale(model, tokenizer, chapter_prompts)
print(generated_tale)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


한 바다에 작은 어부가 살고 있었어요. 그의 이름은 영식이었어요. 어느 날, 영식이는 바다에 나가서 고래를 잡기로 결심했어요. 고래는 그 해변을 어슬렁거리며 헤엄치고 있었어요.
어느 날, 어부는 고래가 자신을 쫓고 있다는 것을 알게 되었어요.
그때, 어부의 등에 타고 있던 작은 고래 한 마리가 갑자기 몸이 떨리고 있다는 걸 알았어요.
그래서 어부를 고래로 착각하고서 바닷속으로 데리고 갔어요.
고래는 고래에게 말했어요.
"저쪽에 큰 고래와 작은 고양이가 있을 거야. 그 고래들을 이용해서 그들을 어요.. 영식이는 고래를 만나게 되었어요. 그러나 그 고래는 사실 고래 백경이었어요. 고래 백경은 영식이에게 부탁을 했어요. "제가 도와드리겠어요!"
영식이가 대답했어요.
"네, 부탁합니다."
그런데 영식은 고래가 너무 고마워서 울음을 뚝 그쳤어요.
"네가 부탁하는 게 뭐가 있니?"
"물론이죠. 제 부탁이 제게는 최고의 선물이에요."
고래는 고래에게 감사의 인사를 전했어요.
그리고 고래와 영식의 결혼을 축하했어요
그들은 함께 행복하게 살았어요.
하지만 어요.. 영식이는 고래 백경의 부탁을 들어주었어요. 그러면서 고래 백경은 영식이와 함께 모험을 떠났어요. 둘은 함께 바다 깊은 곳으로 모험을 떠나고 있었죠.
그들은 서로 다른 점을 존중하고, 다른 이들을 존중하며 함께 즐거운 모험을 즐겼어요.
어느 날, 고래는 자신의 몸에 불을 붙여 백경을 구했어요.
그 바람에 백경이 숨을 쉬지 못했어요.
백경은 고래를 구하기 위해 용감하게 나섰어요.
그는 바다 속에서 고래와 만났어요.
"안녕, 백경아! 함께해줘서 고마워요. 함께라면 어떤 어려움도 극복할 수 있을 거에요."
백경이 어요.. 마지막으로, 영식이와 고래 백경은 함께 어려움을 극복하고 친구가 되었어요. 그리고 그들은 항상 서로를 기억하고 지켜줄 것을 약속했어요. 
"안녕, 백경아! 너의 모험은 정말 즐거웠어!"
백경은 자신의 모험이 끝난 후 활짝 웃었어요.
"이렇게 즐거우면 너에게 많은 것을 줄게."
고래는 웃으며 대답했어요.
"